In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import tqdm
import pdb
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import lightgbm as lgb

import xgboost
from sklearn.preprocessing import LabelEncoder

import random
import os
import pandas

warnings.filterwarnings(action='ignore')

In [18]:
pd.set_option('display.max_row', 50)

path = '../data/'

train = pd.read_csv(path+'ksy_train_rating_10n.csv')
test = pd.read_csv(path+'ksy_test_rating_10n.csv')
submit = pd.read_csv(path + 'sample_submission.csv')

def rmse(real: list, predict: list) -> float:
    pred = np.array(predict)
    return np.sqrt(np.mean((real-pred) ** 2))

SEED = 42
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)

print(train.shape)
print(test.shape)

(306795, 10)
(76699, 10)


In [19]:
tem = train['user_id'].value_counts()
tem = list(tem[tem >= 0].index)
train['id'] = train['user_id'].copy()
train['id'][~train['id'].isin(tem)] = -1
test['id'] = test['user_id'].copy()
test['id'][~test['id'].isin(tem)] = -1

In [20]:
tem = train['isbn'].value_counts()
tem = list(tem[tem >= 0].index)
train['bn'] = train['isbn'].copy()
train['bn'][~train['bn'].isin(tem)] = 1000000000
test['bn'] = test['isbn'].copy()
test['bn'][~test['bn'].isin(tem)] = 1000000000

In [21]:
train['years'] = train['years'].astype('str')
train['fix_age'] = train['fix_age'].astype('str')
train['id'] = train['id'].astype('str')
train['bn'] = train['bn'].astype('str')

test['years'] = test['years'].astype('str')
test['fix_age'] = test['fix_age'].astype('str')
test['id'] = test['id'].astype('str')
test['bn'] = test['bn'].astype('str')

In [22]:
cat_features = ['isbn','book_author','publisher','language', 'category_high', 'fix_location_state', 'bn','years','fix_age','id']

trencode_df = train[cat_features]
teencode_df = test[cat_features]

trencode_df = trencode_df[cat_features].apply(LabelEncoder().fit_transform)
teencode_df = teencode_df[cat_features].apply(LabelEncoder().fit_transform)

trscore_encode_drop = train.drop(cat_features, axis = 1)
tescore_encode_drop = test.drop(cat_features, axis = 1)

train = pd.concat([trscore_encode_drop, trencode_df], axis = 1)
test = pd.concat([tescore_encode_drop, teencode_df], axis = 1)


In [23]:
X_tr, X_val, y_tr, y_val = train_test_split(train.drop(['user_id', 'isbn', 'rating'],axis = 1), train['rating'], test_size=0.2)

In [24]:
# train_ds = lgb.Dataset(X_tr, label = y_tr) 
# test_ds = lgb.Dataset(X_val, label = y_val) 

In [25]:
# params = {'learning_rate': 0.01, 
#           'max_depth': 16, 
#           'boosting': 'gbdt', 
#           'objective': 'regression', 
#           'metric': 'rmse', 
#           'is_training_metric': True, 
#           'num_leaves': 144, 
#           'feature_fraction': 0.9, 
#           'bagging_fraction': 0.7, 
#           'bagging_freq': 5, 
#           'seed':42}

In [26]:
xgb_model = xgboost.XGBRegressor(n_estimators=2000, learning_rate=0.01, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=16)

xgb_model.fit(X_tr,y_tr)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=16, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=2000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             306795 non-null  float64
 1   rating              306795 non-null  float64
 2   isbn                306795 non-null  int64  
 3   book_author         306795 non-null  int64  
 4   publisher           306795 non-null  int64  
 5   language            306795 non-null  int64  
 6   category_high       306795 non-null  int64  
 7   fix_location_state  306795 non-null  int64  
 8   bn                  306795 non-null  int64  
 9   years               306795 non-null  int64  
 10  fix_age             306795 non-null  int64  
 11  id                  306795 non-null  int64  
dtypes: float64(2), int64(10)
memory usage: 28.1 MB


In [28]:
predictions = xgb_model.predict(X_val)
predictions

array([7.4932914, 7.3608828, 6.7186174, ..., 4.9694233, 7.6326346,
       6.940074 ], dtype=float32)

In [29]:
r_sq = xgb_model.score(X_tr, y_tr)
print(mean_squared_error(predictions,y_val)**0.5)

2.207010092327588


In [30]:
submit = pd.read_csv(path + 'sample_submission.csv')
pred = xgb_model.predict(test.drop(['user_id', 'isbn', 'rating'],axis = 1))
test['rating'] = pred
submit['rating'] = test['rating']
submit.to_csv('../data/XGB2.csv', index = False)